In [177]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

# Extracting cloud cover

In [185]:
def extract_cloud_cover(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    _, thresholded = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
    cloud_cover = np.sum(thresholded) / img.size  # Proportion of cloud cover
    return cloud_cover

Preprocessing

In [186]:
def seg_image(img):
    image = cv2.imread(img)

    # Check if image was successfully loaded
    if image is None:
        print(f"Error loading image: {img}")
        return

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply thresholding
    _, thresh = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)

    # Apply Canny edge detection
    edges = cv2.Canny(thresh, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original image
    for contour in contours:
        cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)

    # Show the result
    cv2.imshow('Segmented Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_images_in_directory(directory):
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(directory, filename)  # Fixed line
            seg_image(img_path)

Time series analysis

In [161]:
time_series_data = []
num_groups = 3
num_images_per_group = 15

# Loop through the group and images
for group in range(1, num_groups + 1):
    group_folder = f"group_{group}/"
    
    for img_name in sorted(os.listdir(group_folder))[:num_images_per_group]:
        img_path = os.path.join(group_folder, img_name)
        
        # Extract cloud cover
        cloud_cover = extract_cloud_cover(img_path)
        
        # Extract timestamp from filename
        try:
            date_parts = img_name.split('-')
            if len(date_parts) == 3 and date_parts[2].endswith('.png'):
                date_string = f"20{date_parts[2][:-4]}-{date_parts[1]}-{date_parts[0]}"  # YYYY-MM-DD
                time_series_data.append({'timestamp': date_string, 'group': group, 'cloud_cover': cloud_cover})
            else:
                print(f"Skipping file due to unexpected format: {img_name}")
        except Exception as e:
            print(f"Error processing file {img_name}: {e}")

# Create DataFrame
features_df = pd.DataFrame(time_series_data)
features_df['timestamp'] = pd.to_datetime(features_df['timestamp'], errors='coerce')
features_df.dropna(subset=['timestamp'], inplace=True)  # Drop rows with invalid dates


In [162]:
# Organize data by group
time_series_groups = {}
for group in range(1, num_groups + 1):
    group_data = features_df[features_df['group'] == group].sort_values('timestamp')
    time_series_groups[group] = group_data.set_index('timestamp')['cloud_cover']

In [169]:
predictions = {}
look_back = 5  # Number of past observations to use for predicting the next time step

In [170]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image as keras_image

In [174]:
look_back = 5
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
for group, ts_data in time_series_groups.items():
    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    ts_scaled = scaler.fit_transform(ts_data.values.reshape(-1, 1))  # Change here

    # Prepare sequences with sliding window
    X, y = [], []
    for i in range(len(ts_scaled) - look_back):
        X.append(ts_scaled[i:i + look_back])
        y.append(ts_scaled[i + look_back])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # Reshape for LSTM

    # Build LSTM model with reduced complexity and dropout
    model = Sequential([
    LSTM(50, input_shape=(X.shape[1], 1), return_sequences=False, 
         kernel_regularizer=l2(0.001)),  # Using L2 regularization
    Dropout(0.3),  # Increased dropout rate
    Dense(1)
])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')

    # Train model
    model.fit(X, y, epochs=100, batch_size=2, verbose=1)  # Adjust epochs and batch size
    
    # Forecasting with modified input sequence
    input_seq = ts_scaled[-look_back:]  # Last sequence for prediction
    forecast = []
    for _ in range(5):  # Predict 5 future time steps
        input_seq = np.reshape(input_seq, (1, look_back, 1))
        pred = model.predict(input_seq)
        forecast.append(pred[0, 0])
        input_seq = np.append(input_seq[:, 1:, :], pred.reshape(1, 1, 1), axis=1)  # Update input sequence with new prediction
    
    # Inverse scale the forecast
    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    
    # Create DataFrame for forecast
    last_date = ts_data.index[-1]  # Change here
    forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=5)
    predictions[group] = pd.DataFrame(forecast, index=forecast_dates, columns=['Forecast'])

# Print predictions for each group
for group, forecast_df in predictions.items():
    print(f"Group {group} Forecast:")
    print(forecast_df)


Epoch 1/100
4/4 [==============================] - 1s 2ms/step - loss: 0.3751
Epoch 2/100
4/4 [==============================] - 0s 2ms/step - loss: 0.3460
Epoch 3/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2786
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2096
Epoch 5/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1837
Epoch 6/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1473
Epoch 7/100
4/4 [==============================] - 0s 3ms/step - loss: 0.1489
Epoch 8/100
4/4 [==============================] - 0s 3ms/step - loss: 0.0736
Epoch 9/100
4/4 [==============================] - 0s 3ms/step - loss: 0.0897
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 0.0682
Epoch 11/100
4/4 [==============================] - 0s 3ms/step - loss: 0.0536
Epoch 12/100
4/4 [==============================] - 0s 2ms/step - loss: 0.0421
Epoch 13/100
4/4 [==============================] - 0s 3ms/st